In [4]:
import pandas as pd
import subprocess
import os

In [5]:
workdir_model = '/Users/dongjiacheng/Desktop/coder/mtd/code/metabolic_analysis'

#### 若用户只提交一个转录本数据，则运行下述代码

In [6]:
def run_mt_model_single(input_path, output_path):
    """运行模型

    Args:
        input_path (str): 输入的文件路径
        output_path (str): 输出文件路径
    """

    # 脚本的路径，需要师哥你改路径
    script_path = os.path.join(workdir_model, 'GEM_model_transform_single.py')

    # python GEM_model_transform_single.py --input input_file/model_input.tsv --output output_file/model_output.tsv
    cmd = [
        'Python', script_path,
        '--input', input_path,
        '--output', output_path
    ]
    # 执行脚本并捕获输出
    try:
        result = subprocess.run(cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        return result.stdout
    except subprocess.CalledProcessError as e:
        return e.stderr

# 示例调用
# run_mt_model_single('input_file/model_input.tsv', 'output_file/model_output.tsv')

In [ ]:
import escher
from escher import Builder

def draw_pathway_map_single(model_input_file_path, model_output_file_path, height=900):
    """根据输入的一个转录本数据，运行模型，然后绘制通路图

    Args:
        model_input_file_path (str): 输入的文件路径
        model_output_file_path (str): 输出文件路径
        height (int, optional): 展示的通路图的高度
    
    Returns:
        builder: 返回builder对象
    """
    # 运行模型
    run_mt_model_single(model_input_file_path, model_output_file_path)
    
    # 读取模型的输出文件
    df_reaction = pd.read_csv(model_output_file_path,sep='\t')
    df_reaction = df_reaction.iloc[:, [0, 2]]
    df_reaction['flux'] = df_reaction['flux'].round(6)

    # 将reaction_data_df转换为字典
    df_reaction_dict = df_reaction.set_index('reactionid').to_dict()['flux']

    # 前期配置
    escher.rc['never_ask_before_quit'] = True
    builder = Builder()
    builder.hide_secondary_metabolites = True
    builder.hide_all_labels = True

    # 设置builder的显示
    builder.reaction_scale = [
        { 'type': 'min', 'color': '#000000', 'size': 12 },
        { 'type': 'median', 'color': '#ffffff', 'size': 20 },
        { 'type': 'max', 'color': '#ff0000', 'size': 25 }
    ]
    builder.reaction_scale_preset = 'GaBuRd'
    builder.reaction_scale = [
        {k: v * 3 if k == 'size' else v for k, v in x.items()}
        for x in builder.reaction_scale
    ]

    # 通路背景文件路径
    map_path = os.path.join(workdir_model,'modespoth_20230328_map.json')

    # 加载builder
    builder = Builder(
        height=height, # 设置高度
        map_name=None,
        map_json=map_path,
        model_name='e_coli_core',
        reaction_data=df_reaction_dict  # 传递反应数据
    )

    # 保存为html文件
    html_path = os.path.join(workdir_model, 'output_file/pathway_map_single.html')
    builder.save_html(html_path)

    return builder

# 示例调用
draw_pathway_map_single('input_file/model_input.tsv', 'output_file/model_output.tsv')

#### 用户提交两个转录本数据，则运行下述代码

In [8]:
import subprocess
import os

def run_mt_model(input_wt_path, input_treatment_path, output_path):
    """根据输入的两个转录本数据运行模型

    Args:
        input_wt_path (str): 输入的对照组文件路径
        input_treatment_path (str): 输入的处理组文件路径
        output_path (str): 输出文件路径
    
    """
    # 脚本的路径，需要师哥你改workdir_model的路径
    script_path = os.path.join(workdir_model, 'GEM_model_transform_paired.py')

    # python GEM_model_transform_paired.py --input_c input_file/model_input_control.tsv --input_t input_file/model_input_treatment.tsv --output output_file/model_output_difference.tsv
    cmd = [
        'Python', script_path,
        '--input_c', input_wt_path,
        '--input_t', input_treatment_path,
        '--output', output_path
    ]

    # 执行脚本并捕获输出
    try:
        result = subprocess.run(cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        return result.stdout
    except subprocess.CalledProcessError as e:
        return e.stderr

# 示例调用
# run_mt_model('input_file/model_input_control.tsv', 
#              'input_file/model_input_treatment.tsv', 
#              'output_file/model_output_difference.tsv')

In [ ]:
def draw_pathway_map(model_input_wt_file_path, model_input_treatment_file_path, model_output_file_path, height=900):
    """绘制代谢通路图,并保存为html文件

    Args:
        model_input_wt_file_path (str): wt数据输入文件路径
        model_input_treatment_file_path (str): treatment数据输入文件路径
        model_output_file_path (str): 模型输出文件路径
        height (int, optional): 图片高度. Defaults to 900.
    """
    # 运行模型
    run_mt_model(model_input_wt_file_path, model_input_treatment_file_path, model_output_file_path)
    
    # 读取模型的输出文件
    df_reaction = pd.read_csv(model_output_file_path,sep='\t')
    df_reaction = df_reaction.iloc[:, [0, 2, 4]]

    # 计算两种条件下通量的差异
    df_reaction['flux_difference'] = df_reaction['flux_df2'] - df_reaction['flux_df1']
    # 保留两位小数
    df_reaction['flux_difference'] = df_reaction['flux_difference'].round(6)

    # 转换为字典
    df_reaction_dict = df_reaction.set_index('reactionid')['flux_difference'].to_dict()

    # 前期配置
    escher.rc['never_ask_before_quit'] = True
    builder = Builder()
    builder.hide_secondary_metabolites = True
    builder.hide_all_labels = True

    # 设置builder的显示
    builder.reaction_scale = [
        { 'type': 'min', 'color': '#000000', 'size': 12 },
        { 'type': 'median', 'color': '#ffffff', 'size': 20 },
        { 'type': 'max', 'color': '#ff0000', 'size': 25 }
    ]
    builder.reaction_scale_preset = 'GaBuRd'
    builder.reaction_scale = [
        {k: v * 3 if k == 'size' else v for k, v in x.items()}
        for x in builder.reaction_scale
    ]

    # 通路背景文件路径
    map_path = os.path.join(workdir_model,'modespoth_20230328_map.json')

    # 加载builder
    builder = Builder(
        height=height, # 设置高度
        map_name=None,
        map_json=map_path,
        model_name='e_coli_core',
        reaction_data=df_reaction_dict  # 传递你的反应数据
    )

    # 保存为html文件
    html_path = os.path.join(workdir_model, 'output_file/pathway_map_paired.html')
    builder.save_html(html_path)

    return builder

# 示例调用
draw_pathway_map('input_file/model_input_control.tsv', 
                 'input_file/model_input_treatment.tsv', 
                 'output_file/model_output_difference.tsv')